### Installation

Install dependencies 

In [25]:
%pip install langchain
%pip install openai
%pip install PyPDF2
%pip install faiss-cpu
%pip install boto3
%pip install tiktoken 
%pip install unstructured


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 

### Load Required Packages

In [29]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS
from langchain.document_loaders import OnlinePDFLoader
from urllib.request import Request, urlopen
import io


from io import BytesIO

### Open AI Setup

In [3]:
import os

with open('../openai_secret.txt', 'r') as f:
    first_line = f.readline()
    os.environ["OPENAI_API_KEY"] = first_line    


### S3 Connection Setup

In [19]:
import boto3

# Set up the S3 client
s3 = boto3.client('s3')

bucket_name = "engageli-matan-media-manager-matan-us-east-1"
prefix = "matan/gpt-test/"

### Retrive the files from S3

In [20]:
documents = []

def getFilesFromFolder(folder_name = ""):
    prefix += folder_name    
    # List all objects in the S3 bucket location
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

    # Get the URLs of all objects in the folder
    object_urls = []
    for obj in response['Contents']:
        object_url = f"https://{bucket_name}.s3.amazonaws.com/{obj['Key']}"
        object_urls.append(object_url)

    # Print the URLs of all objects in the folder
    documents.append(object_urls)
    
def getExampleFile(file_url):
    documents.append(file_url)

### PDF Reader

In [31]:
def readMultipleFiles():
    ## TODO
    x = 1

def readAndExtractOneFileFromLocal():
    reader = PdfReader("../assets/pre-processing/PDFs/2023HydrogenInvestorDayTranscript.pdf")

    raw_text = ''
    for i, page in enumerate(reader.pages):
        text = page.extract_text()
        if text:
            raw_text += text
    return raw_text

def get_pdf_from_url(url):
    """
    :param url: url to get pdf file
    :return: PdfFileReader object
    """
    remote_file = urlopen(Request(url)).read()
    memory_file = io.BytesIO(remote_file)
    pdf_file = PdfReader(memory_file)
    return pdf_file

def extract_text_from_pdf(pdf):
    raw_text = ''
    for i, page in enumerate(pdf.pages):
        text = page.extract_text()
        if text:
            raw_text += text
    return raw_text


In [32]:
pdf = get_pdf_from_url("https://engageli-matan-media-manager-matan-us-east-1.s3.amazonaws.com/matan/gpt-test/2023+Hydrogen+Investor+Day+Transcript+(2023.03.09).pdf")
raw_text = extract_text_from_pdf(pdf)

In [27]:
raw_text = readAndExtractOneFileFromLocal("https://engageli-matan-media-manager-matan-us-east-1.s3.amazonaws.com/matan/gpt-test/2023+Hydrogen+Investor+Day+Transcript+(2023.03.09).pdf")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/matan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.


AttributeError: 'list' object has no attribute 'pages'

In [33]:
# We need to split the text that we read into smaller chunks so that during information retreival we don't hit the token size limits. 

text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [34]:
texts[24]

"it really is all 3. Basically, it's the same solution but a pretty different set of buyers and a very different\ncompetitive landscape when we think about who's playing in each of those verticals.\nAnd then the final side of this triangle, which you're all probably like well, what about that? It's, how do\nwe connect learners to jobs? If we're saying that there's a demand for skills that has been unprecedented\nand people who have those skills can now work remotely in other geographic regions, wouldn't it be really\nvaluable to help learners who have skills, find job opportunities and help employers who are looking for\ntalented low-cost employees in emerging talent pools get connected to the learners who have those skills.\nAnd so this is kind of where we're going to be focusing for the next 3 years or so is connecting these\nlearners to job opportunities.\nNow since the IPO, we've grown a number of these things pretty nicely, 40 million -- about 40 million"

In [35]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [36]:
docsearch = FAISS.from_texts(texts, embeddings)

In [37]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [38]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [39]:
query = "who is the head of investor relations?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Cam Carey'

In [15]:
query = "what are the 3 segments of the Coursera segment?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The 3 segments of the Coursera platform are Consumer, Enterprise, and Degrees.'

In [16]:
query = "when the adjusted EBIDA is planned to be positive?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' In 2024.'